## TVB TimeSeries Widget

In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet -e ..
%pip install tvb-data

---

### This notebook is dedicated to showcasing the TimeSeries widget using TVB data

### Intial setup

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from tvbwidgets.api import TimeSeriesWidget
from IPython.core.display_functions import display

### Generate TVB data

 #### A --  This TS has only 1 state variable and 1 mode:

In [ ]:
from tvbwidgets.tests.ts_generator import generate_ts_with_stimulus
tsr1 = generate_ts_with_stimulus()
tsr1

#### B -- Generate TVB TS data with more modes:

In [ ]:
from tvbwidgets.tests.ts_generator import generate_ts_with_mode_and_sv
#tsr2 = generate_ts_with_mode_and_sv()
#tsr2

### Plot data using our TS widget

In [ ]:
tsw = TimeSeriesWidget()
tsw.add_datatype(tsr1)
display(tsw)